In [1]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch
import yaml

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Setup complete ✅ (36 CPUs, 376.5 GB RAM, 0.5/188.3 GB disk)


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
model = YOLO('yolov8n.pt')

In [4]:
# TRAIN the model on our dataset (data.yml config file) for 10 epochs
results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=50,
   batch=32,
   cos_lr=True,
   #dropout=0.6,
   name='yolov8n_custom')

New https://pypi.org/project/ultralytics/8.0.207 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom37, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=Fals

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/train
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/train


train: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/train/labels... 6030 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6030/6030 [00:13<00:00, 459.20it/s]
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_0.0_448_448.png: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_2.0_448_448.png: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_2.0_0.0_448_448.png: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/train/images/global_birds_pfe

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/val
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/val


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/val/labels... 849 images, 0 backgrounds, 0 corrupt: 100%|██████████| 849/849 [00:02<00:00, 418.97it/s]
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_0.0_0.0_448_448.png: 176 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_0.0_2.0_448_448.png: 177 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_2.0_0.0_448_448.png: 177 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/val/images/global_birds_pfeifer_Rzepecki Island

## **Validation on test subset**

In [5]:
# EVALUATE the model's performance on the test set
results = model.val(split='test', save_json=True)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/labels... 1825 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1825/1825 [00:04<00:00, 439.66it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [00:19<00:00,  3.00it/s]
                   all       1825      25818      0.652      0.643      0.534       0.21
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 3.0ms postprocess per image
Saving runs/detect/yolov8n_custom372/predictions.json...
Results saved to runs/detect/yolov8n_custom372


In [ ]:
# Perform MODEL PREDICTION on the test set
model.predict(
    model = 'runs/detect/yolov8n_custom93',
    source ='/gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images',
    conf = 0.25
)

In [ ]:
# Save PREDICTIONS PREVIEWS

import visualization_utils as visutils
import os

#results = model(source = '/gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images/')
threshold = 0.5
category_id_to_name = {0: 'bird'}
detection_boxes = []

for result in results:
    im_array = result.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    #im.show()  # show image
    im.save('/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/yolov8n_custom12/predictions/' + os.path.basename(result.path).split('.png')[0] + '.jpg')  # save image


## **Validation on test subset per dataset**

In [6]:
database1_source = ['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins',
                    'global_birds_mckellar', 'global_birds_newmexico', 
                    'global_birds_pfeifer', 'uav_thermal_waterfowl']

In [7]:
database1_source = ['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins',
                    'global_birds_mckellar', 'global_birds_newmexico', 
                    'global_birds_pfeifer']

In [7]:
#model = YOLO('runs/detect/yolov8n_custom34/weights/best.pt')
database1_source = ['global_birds_poland', 'global_birds_newmexico', 'global_birds_pfeifer', 'global_birds_mckellar']


In [8]:
for dataset in database1_source:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    model.val(split='test', save_json=True, )


Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_poland/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_poland/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_poland/labels...:   0%|          | 0/218 [00:00<?, ?it/s]

val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_poland/labels... 218 images, 0 backgrounds, 0 corrupt: 100%|██████████| 218/218 [00:00<00:00, 371.07it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_poland/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.41it/s]
                   all        218        873      0.666      0.805      0.644      0.245
Speed: 0.3ms preprocess, 1.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Saving runs/detect/yolov8n_custom373/predictions.json...
Results saved to runs/detect/yolov8n_custom373
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_palmyra/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_palmyra/images


...

val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_mckellar/labels... 244 images, 0 backgrounds, 0 corrupt: 100%|██████████| 244/244 [00:00<00:00, 384.33it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_mckellar/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.75it/s]
                   all        244        748      0.352      0.457      0.332      0.142
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Saving runs/detect/yolov8n_custom376/predictions.json...
Results saved to runs/detect/yolov8n_custom376
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_newmexico/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_newmexico/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_newmexico/labels... 176 images, 0 backgrounds, 0 corrupt: 100%|██████████| 176/176 [00:00<00:00, 368.35it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_newmexico/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:04<00:00,  1.44it/s]
                   all        176       4007      0.735      0.538      0.621      0.239
Speed: 0.3ms preprocess, 1.6ms inference, 0.0ms loss, 10.6ms postprocess per image
Saving runs/detect/yolov8n_custom377/predictions.json...
Results saved to runs/detect/yolov8n_custom377
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_pfeifer/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_pfeifer/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_pfeifer/labels... 723 images, 0 backgrounds, 0 corrupt: 100%|██████████| 723/723 [00:01<00:00, 525.94it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_6_datasets_no_background/test/global_birds_pfeifer/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:10<00:00,  2.14it/s]
                   all        723      15804      0.803      0.818      0.738      0.309
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 5.9ms postprocess per image
Saving runs/detect/yolov8n_custom378/predictions.json...
Results saved to runs/detect/yolov8n_custom378
